The purpose of this project is to be able to track a known object (the paper) in a video and use its corners to estimate the pose of the camera

The pipeline is as follows
1. Perform edge deteection on each frame
2. Generate a hough map in polar coordinates, the peaks of ewhick will describe the equation of the line normal to the edge in the image, coming from the origin
3. Convert the polar equation of the line back to cartesian coordinates for each line. Use these line equations to find the corners in the image
4. Using the locations of the real world corners, the location of the corners in the image, and the intrinsic matrix of the camera, find the three rotational vectors and the translation vector


In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy
from copy import deepcopy
from scipy.spatial.transform import Rotation   

In [2]:
scale = 0.5

# following the formula of x = KR [I_3 | -C] X
# using the intrinsic matrix and the detected image points, we
# can calculate the rotation and translation vectors with a generated homography

k = np.array(
    [
        [1380, 0, 946],
        [0, 1380, 527],
        [0, 0, 1]
    ]
)

scaling_matrix = np.array(
    [
        [scale, 0, 0],
        [0, scale, 0],
        [0, 0, 1]
    ]
)

k = scaling_matrix @ k
print(k)
k_inv = np.linalg.inv(k)




[[690.    0.  473. ]
 [  0.  690.  263.5]
 [  0.    0.    1. ]]


In [3]:
def gen_mask(frame):

    # find the maks of the ball
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (9,9), 0)
    mask = cv2.inRange(gray, (180), (255))

    kernel = np.ones((5,5),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask[:, :int(frame.shape[1]*0.2)] = 0
    
    return mask

In [4]:
def min_max_scale(frame):
    
    min_val = np.min(frame)
    max_val = np.max(frame)

    frame = (frame - min_val) / (max_val - min_val)
    
    return frame


In [5]:
def canny_edge(frame):
    
    sobel_x = 1/8*np.array(
        [
            [-1, 0, 1],
            [-2, 0, 2],
            [-1, 0, 1]
        ]
    )

    sobel_y = 1/8*np.array(
        [
            [1, 2, 1],
            [0, 0, 0],
            [-1, -2, -1]
        ]
    )
    
    x_grad = scipy.signal.correlate(frame, sobel_x, 'same')
    y_grad = scipy.signal.correlate(frame, sobel_y, 'same')
    
    
    grad_intensity = (np.sqrt(x_grad**2 + y_grad**2))
    grad_intensity = min_max_scale(grad_intensity)
    grad_intensity[grad_intensity < 0.5] = 0
    grad_direction = np.arctan2(x_grad, y_grad)
    
    return grad_intensity, grad_direction

In [6]:
def gen_hough_map(edge_frame):

    # initialize hough map
    shape = (int(np.sum(edge_frame.shape)), 180)
    offset = shape[0]//2
    hough_map = np.zeros(shape = shape)

    # find the indeces of the nonzero elements
    nonzero_y, nonzero_x = np.nonzero(edge_frame)
    num_nonzero = len(nonzero_y)


    for idx in range(num_nonzero):
        
        xi = nonzero_x[idx]
        yi = nonzero_y[idx]
        
        for t in range(180):
            
            # calculate the depth of the line, then offset
            d = int(xi*np.cos(t*np.pi/180) + yi*np.sin(t*np.pi/180))
            d = d - 1 + offset
            
            # update this tick for the hough map 
            hough_map[d, t] += 1
        
        
    hough_map = min_max_scale(hough_map)
    return hough_map


In [7]:
def get_hough_peaks(hough_map):
    
    height, width = hough_map.shape
    
    max_intensity = -np.inf
    th1 = 0
    th2 = 0
    
    # iterate over the columns and find the ones that have max values 90 degrees apart
    # do this to find our thetas
    for theta in range(width//2):
        col1 = hough_map[:, theta]
        col2 = hough_map[:, theta+90]
        
        combined_intensity = max(col1) + max(col2)
        if combined_intensity > max_intensity:
            max_intensity = combined_intensity
            
            th1 = theta
            th2 = theta+90
       
    
    peaks = []
    # for each of the thetas, sort, and pick the top two in each column
    # which are at least 10 pixels apart
    for th in [th1, th2]:
        col = hough_map[:, th]
        indeces = np.argsort(col)[::-1]

        # choose the first value in the sorted list as peak 1
        # then move theough the other sorted values and if the next highest value is 
        # at least a certain distance away, set it as the second peak in the column
        peak1 = [indeces[0], th]
        peak2 = None
        for i in indeces:
            if np.abs(i-indeces[0]) >= 10:
                peak2 = [i, th]
                break
            
        peaks.append(peak1)
        peaks.append(peak2)
        
        
    # unoffset from when we generated the hough map
    for p in peaks:
        p[0] = p[0] + 1 - height//2
        
    return peaks



In [8]:
def lines_to_corner(line_param1, line_param2):
    
    m1, b1 = line_param1
    m2, b2 = line_param2
    
    x = (b2-b1)/(m1-m2)
    y = m1*x + b1
    
    return [int(x), int(y)]

In [9]:
def corners_from_peaks(frame, peaks):


    height, width, _ = frame.shape
    lines = []

    for p in peaks:
        d = p[0]
        th = p[1]*np.pi/180
        
        m = -1*np.cos(th)/np.sin(th)
        b = d/np.sin(th)
        
        lines.append((m, b))

        xs = []
        ys = []
        
        for x in range(width):
            y = m*x + b
            if y >= 0 and y <= height:
                xs.append(int(x))
                ys.append(int(y))
                
        start_pt = ((xs[0]), ys[0])
        end_pt = (xs[-1], ys[-1])
        
        frame = cv2.line(frame, start_pt, end_pt, [0,255,0], 2)

    #find corners from line equations
    c1 = lines_to_corner(lines[0], lines[2])    
    c2 = lines_to_corner(lines[0], lines[3])    
    c3 = lines_to_corner(lines[1], lines[2])    
    c4 = lines_to_corner(lines[1], lines[3])    
    
    # aggregate and sort by y
    corners = [c1, c2, c3, c4]
    corners.sort(key = lambda x: x[1])

    return np.array(corners)
        

In [10]:
def find_h(image_points):

    world_points = np.array(
        [
            [0,     0],
            [0.279, 0],
            [0,     0.216],
            [0.279, 0.216],
        ]
    )
    
    
    a_mat = np.array([])
    
    rows = []
    
    for i in range(len(image_points)):
        xc = image_points[i][0]
        yc = image_points[i][1]
        
        Xw = world_points[i][0]
        Yw = world_points[i][1]
        
        rows.append([Xw, Yw, 1, 0,  0,  0, -xc*Xw, -xc*Yw, -xc])
        rows.append([0,  0,  0, Xw, Yw, 1, -yc*Xw, -yc*Yw, -yc])
        
    
    a_mat = np.array(rows)
    aTa = a_mat.T@a_mat
    eigvals, eigvecs = np.linalg.eig(aTa)
    
    p = eigvecs[:, np.argmin(eigvals)]
    p = p.reshape(3, 3)
    return p

In [11]:
def get_camera_frame(image_point):
    
    h = find_h(image_point)

    extr = k_inv@h

    r1 = extr[:, 0]
    r2 = extr[:, 0]
    r3 = np.cross(r1, r2)

    r = np.array([r1, r2, r3]).T

    scipy_rot = Rotation.from_matrix(r)
    rpy = scipy_rot.as_euler('xyz', degrees=True)
    c = extr[:, -1]
    
    l = (np.linalg.norm(r1) + np.linalg.norm(r2))/2
    t = extr[:, 2]/l
    
    return rpy, t

In [12]:
def write_telem_to_frame(frame, rpy, t):
    
    font = cv2.FONT_HERSHEY_SIMPLEX 
  
    r = np.round(rpy[0], 4)
    p = np.round(rpy[1], 4)
    y = np.round(rpy[2], 4)
  
    tx = np.round(t[0], 4)
    ty = np.round(t[1], 4)
    tz = np.round(t[2], 4)

    cv2.putText(frame,  
                f'RPY = {r, p, y}',  
                (50, 50),  
                font, 1,  
                (0, 0, 0),  
                2,  
                cv2.LINE_4) 
    
    
    cv2.putText(frame,  
                f'T = {tx, ty, tz}',  
                (50, 100),  
                font, 1,  
                (0, 0, 0),  
                2,  
                cv2.LINE_4) 
    
    return frame

In [13]:
def process_video(scale = 1):

    cap = cv2.VideoCapture('project2.avi')

    frames = []
    image_pts = []
    # masks = []
    # edges = []
    # hough_maps = []
    
    # this reads the video, captures frames, collects 
    # and plots the coordinates of the center of the ball 
    while cap.isOpened():
        ret, frame = cap.read()

        # if frame is read correctly ret is True
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        height, width, _ = frame.shape
        frame = cv2.resize(frame, (int(width*scale), int(height*scale)))
        
        mask = gen_mask(frame)
        gradient_intensity, gradient_direction = canny_edge(mask)
        hough_map = gen_hough_map(gradient_intensity)
        peaks = get_hough_peaks(hough_map)
        corners = corners_from_peaks(frame,peaks)        
        
        rpy, t = get_camera_frame(corners)
                
        for c in corners:
            frame = cv2.circle(frame, (c[0], c[1]), 5, [0,0,255], -1)

        frame = write_telem_to_frame(frame, rpy, t)

        frames.append(frame)
        image_pts.append(corners)        
        # masks.append(mask)
        # edges.append(gradient_intensity)
        # hough_maps.append(hough_map)        
    
        cv2.imshow('mask', frame)
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    return frames, image_pts
# , masks, edges, hough_maps



In [14]:
def write_video(fname, frames_to_write, single_channel):


    size = (frames_to_write[0].shape[1], frames_to_write[0].shape[0]) 

    # Below VideoWriter object will create 
    # a frame of above defined The output  
    # is stored in 'filename.avi' file. 
    result = cv2.VideoWriter(fname,  
                            cv2.VideoWriter_fourcc(*'MJPG'), 
                            20, size) 
    
    for frame in frames_to_write: 
        if single_channel:
            frame = min_max_scale(frame)*255
            frame = frame.astype(np.uint8)
            result.write(cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR))
        else:
            result.write(frame) 
        
    result.release()

In [15]:

frames, image_points = process_video(scale)
# masks, edges, hough_maps = process_video(0.5)

write_video('frames.avi', frames, False)
# write_video('masks.avi', masks, True)
# write_video('edges.avi', edges, True)

# for i in range(len(hough_maps)):
#     hough_maps[i] = cv2.resize(hough_maps[i], (700,700))
    
# write_video('hough_lines.avi', hough_maps, True)



Can't receive frame (stream end?). Exiting ...
